## Категории в данных о недвижимости

In [ ]:
# собираем все предыдующие преобразования таблицы melb_df в одном месте
# для удобства
import pandas as pd


melb_data = pd.read_csv('data/melb_data_ps.csv', sep=',')
melb_df = melb_data.copy()


# сразу заменяю таблицу
melb_df.drop(['index', 'Coordinates'], axis=1, inplace=True)

# создадим переменную которая будет содержать информацию о всех переменных
total_rooms = melb_df['Rooms'] + melb_df['Bathroom'] + melb_df['Bedroom']

# создаем новый признак
melb_df['MeanRoomsArea'] = melb_df['BuildingArea'] / total_rooms

# создаем еще один новый признак 
melb_diff = melb_df['BuildingArea'] - melb_df['Landsize']
# сумму двух площадей будем использовать для расчета коэфициэнта площади 
melb_sum = melb_df['BuildingArea'] + melb_df['Landsize']
# создаем переменную с расчитанной информацией
melb_df['AreaRatio'] = melb_diff / melb_sum

melb_df['Date'] = pd.to_datetime(melb_df['Date'], dayfirst=True)

melb_df['MonthSale'] = melb_df['Date'].dt.month
# узнаем в каком месяце было больше всего продаж
melb_df['MonthSale'].value_counts(normalize=True)

# создаем новый признак который содержит информацию возрастве здания на момент 
# продажи
melb_df['AgeBuilding'] = melb_df['Date'].dt.year - melb_df['YearBuilt']

# удаляем столбец
melb_df = melb_df.drop('YearBuilt', axis=1)


# на вход поступает строка одной позиции с адресом
def get_street_type(address):
    # создаем список географических пометок
    exclude_list = ['S', 'N', 'W', 'E']
    # разделяем строку по пробелам
    address_list = address.split(' ')
    # узнаем какой последний элемент списка и заносим его в переменную
    street_type = address_list[-1]
    # проверяем на то что это географическая пометка и создаем условие
    if street_type in exclude_list:
        # переопределяем на второй с конца элемент
        street_type = address_list[-2]
    # если все в порядке возвращаем тип улицы
    return street_type

# используем функцию на столбец и получаем отдельные типы улиц в переменной
street_types = melb_df['Address'].apply(get_street_type)

# из обьекта типа Series выделим десять самых популяряных
popular_stypes = street_types.value_counts().nlargest(10).index

# переопределяем и чистим обьект типа Series
street_types = street_types.apply(lambda x: x if x != 'Avenue' else 'Av')
street_types = street_types.apply(lambda x: x if x != 'Boulevard' else 'Bvd')
street_types = street_types.apply(lambda x: x if x != 'Parade' else 'Pde')

# применим lambda-функцию на обьект Series со всеми подтипами
# а те кто не входит в список популярных меняем на other
melb_df['StreetType'] = street_types.apply(lambda x: x if x in popular_stypes \
    else 'other')

# удаляем ненужный признак
melb_df = melb_df.drop('Address', axis=1)


# приводим к формату datetime
melb_df['Date'] = pd.to_datetime(melb_df['Date'], dayfirst=True)
# создаем новый признак с днями недели
melb_df['WeekdaySale'] = melb_df['Date'].dt.dayofweek
# создаю маску для посчета через .shape
mask_one = melb_df['WeekdaySale'] == 6
mask_two = melb_df['WeekdaySale'] == 5
# помещаю ответ в переменную
weekend_count = melb_df[mask_one | mask_two].shape[0]


# создаем функцию которую применим на столбец
def get_weekend(weekday):
    # создаем список цифор, которые будут являтся буднями
    exclude_list = [0, 1, 2, 3, 4]
    if weekday in exclude_list:
        return 0
    else:
        return 1
    
# создаем новый признак
melb_df['Weekend'] = melb_df['WeekdaySale'].apply(get_weekend)


# создаем список из 49 нужных значений
top_sellar = melb_df['SellerG'].value_counts().nlargest(49).index

# перкладываем обьект Series в переменную
top_sellar_col = melb_df['SellerG'].apply(lambda x: x if x in top_sellar \
    else 'other')
# удаляю оригинальный признак
melb_df.drop('SellerG', axis=1)
# заменяю его очищенным
melb_df['SellerG'] = top_sellar_col

# с помощью метода .reaname_categories переименовываем значения столбца
melb_df['Type'] = melb_df['Type'].cat.rename_categories({
    'u': 'unit',
    't': 'townhouse',
    'h': 'house'
})

# добавляем в интерисующий нас признак новый новую категорию
melb_df['Type'] = melb_df['Type'].cat.add_categories('flat')
# пересоздаем ряд
new_categories_type = pd.Series(['unit', 'house', 'flat', 'flat', 'house'])
# теперь все категории отображаются
new_categories_type = new_categories_type.astype(melb_df['Type'].dtype)

popular_list = melb_df['Suburb'].value_counts().nlargest(119).index

melb_df['Suburb'] = melb_df['Suburb'].apply(lambda x: x if x in popular_list \
    else 'other')
melb_df['Suburb'] = melb_df['Suburb'].astype('category')


In [18]:
# создаем пустой список для хранения кортежей
unique_list = []
# расматриваем имя каждого признака
for col in melb_df.columns:
    # создаем кортеж 
    item = (col, melb_df[col].nunique(), melb_df[col].dtypes)
    # заполняем список
    unique_list.append(item)
# создаем вспомогательную таблицу и сортируем ее
unique_counts = pd.DataFrame(
    unique_list,
    columns = ['Column_Name', 'Num_Unique', 'Type']
).sort_values(by='Num_Unique', ignore_index=True)
# выводим на экран
display(unique_counts)

,Column_Name,Num_Unique,Type
0,Weekend,2,int64
1,Type,3,object
2,WeekdaySale,5,int32
3,Method,5,object
4,Regionname,8,object
5,Rooms,9,int64
6,Bathroom,9,int64
7,Car,11,int64
8,StreetType,11,object
9,Bedroom,12,int64


## Тип данных Category

In [20]:
# узнаем количество занимаемой памяти таблице
display(melb_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 26 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Suburb         13580 non-null  object        
 1   Rooms          13580 non-null  int64         
 2   Type           13580 non-null  object        
 3   Price          13580 non-null  float64       
 4   Method         13580 non-null  object        
 5   SellerG        13580 non-null  object        
 6   Date           13580 non-null  datetime64[ns]
 7   Distance       13580 non-null  float64       
 8   Postcode       13580 non-null  int64         
 9   Bedroom        13580 non-null  int64         
 10  Bathroom       13580 non-null  int64         
 11  Car            13580 non-null  int64         
 12  Landsize       13580 non-null  float64       
 13  BuildingArea   13580 non-null  float64       
 14  CouncilArea    12211 non-null  object        
 15  Lattitude      1358

None

In [23]:
# создаем список, который не должен попадать под преобразование в тип Category
cols_to_exclude = ['Date', 'Rooms', 'Bedroom', 'Bathroom', 'Car']
# создадим условие для преобразование в тип category те признаки в\
# в которых количество уникальных значений меньше 150
max_unique_count = 150
# циклом проходимся по всем призначкам
for col in melb_df.columns:
    # создаем условие для фильтрации
    if melb_df[col].nunique() < max_unique_count and col not in \
    cols_to_exclude:
        # меняем тип данныз самым легким способом
        melb_df[col] = melb_df[col].astype('category')

display(melb_df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 26 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Suburb         13580 non-null  object        
 1   Rooms          13580 non-null  int64         
 2   Type           13580 non-null  category      
 3   Price          13580 non-null  float64       
 4   Method         13580 non-null  category      
 5   SellerG        13580 non-null  category      
 6   Date           13580 non-null  datetime64[ns]
 7   Distance       13580 non-null  float64       
 8   Postcode       13580 non-null  int64         
 9   Bedroom        13580 non-null  int64         
 10  Bathroom       13580 non-null  int64         
 11  Car            13580 non-null  int64         
 12  Landsize       13580 non-null  float64       
 13  BuildingArea   13580 non-null  float64       
 14  CouncilArea    12211 non-null  category      
 15  Lattitude      1358

None

### .cat.categories

In [24]:
# смотрим на список унмкальных значений при помощи аксессора .cat
# и его атрибута .categories
print(melb_df['Regionname'].cat.categories)

Index(['Eastern Metropolitan', 'Eastern Victoria', 'Northern Metropolitan',
       'Northern Victoria', 'South-Eastern Metropolitan',
       'Southern Metropolitan', 'Western Metropolitan', 'Western Victoria'],
      dtype='object')


### .cat.codes

In [25]:
# смотрм на числовой код при помощи атрибута .categories
display(melb_df['Regionname'].cat.codes)

0        2
1        2
2        2
3        2
4        2
        ..
13575    4
13576    6
13577    6
13578    6
13579    6
Length: 13580, dtype: int8

### .cat.reaname_categories()

In [27]:
# с помощью метода .reaname_categories переименовываем значения столбца
melb_df['Type'] = melb_df['Type'].cat.rename_categories({
    'u': 'unit',
    't': 'townhouse',
    'h': 'house'
})

display(melb_df['Type'])

0        house
1        house
2        house
3        house
4        house
         ...  
13575    house
13576    house
13577    house
13578    house
13579    house
Name: Type, Length: 13580, dtype: category
Categories (3, object): ['house', 'townhouse', 'unit']

In [30]:
# видим, что новые категории будут типа Nan
new_categories_type = pd.Series(['unit', 'house', 'flat', 'flat', 'house'])
# преобразуем новый ряд при помощи признака с типом данных categories
new_categories_type = new_categories_type.astype(melb_df['Type'].dtype)

display(new_categories_type)

0     unit
1    house
2      NaN
3      NaN
4    house
dtype: category
Categories (3, object): ['house', 'townhouse', 'unit']

### .cat.add_categories

In [31]:
# добавляем в интерисующий нас признак новый новую категорию
melb_df['Type'] = melb_df['Type'].cat.add_categories('flat')
# пересоздаем ряд
new_categories_type = pd.Series(['unit', 'house', 'flat', 'flat', 'house'])
# теперь все категории отображаются
new_categories_type = new_categories_type.astype(melb_df['Type'].dtype)

display(new_categories_type)


0     unit
1    house
2     flat
3     flat
4    house
dtype: category
Categories (4, object): ['house', 'townhouse', 'unit', 'flat']